In [33]:
import cv2 as cv
print("Current version of openCV=", cv.__version__)
import numpy as np
import os
import glob

Current version of openCV= 4.1.1


In [36]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 6
points_per_column = 10
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./calibration/pinhole/*.png')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (points_per_row,points_per_column), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (points_per_row,points_per_column), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [37]:
#flags = None#cv.CALIB_RATIONAL_MODEL
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], 
                                                  cameraMatrix=None, distCoeffs=None)
print("ret: ", ret)
print("K matrix: ", mtx)
print("D distortions: ", dist)
#print("rotation vecs: ", rvecs)
print("translation vecs: ", tvecs)


ret:  0.2119902232721743
K matrix:  [[1.08222078e+03 0.00000000e+00 6.45688878e+02]
 [0.00000000e+00 1.08587949e+03 3.57887541e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
D distortions:  [[ 0.02451276  0.04372337 -0.00407773  0.0011938  -0.12879966]]
translation vecs:  [array([[ 1.01603615],
       [ 0.82690991],
       [34.89787641]]), array([[ 8.03480057],
       [-0.74597336],
       [30.31138083]]), array([[-6.24642278],
       [-5.02851353],
       [17.12632652]]), array([[ 6.01873571],
       [ 0.8752473 ],
       [34.94598391]]), array([[-0.58916855],
       [-4.65208094],
       [16.72415943]]), array([[ 4.26954747],
       [-5.64863278],
       [20.16497454]]), array([[ 2.61740492],
       [-2.70066219],
       [21.94410632]]), array([[-7.91072829],
       [-3.40168314],
       [19.62006699]]), array([[-5.11224734],
       [ 1.25004218],
       [34.28937066]]), array([[-8.66113782],
       [-4.08874822],
       [19.40342808]]), array([[ 4.38486429],
       [-4.985388

In [38]:
img_dir = 'calibration/pinhole/'
#img_dir = 'fisheye_images/'
img_name  = '{}.png'
#dst_dir = 'calibration/pinhole_calibrated/'
dst_dir = 'calibration/pinhole_calibrated/'
for img_id in range(len(images)):
    img = cv.imread(img_dir + img_name.format(img_id))
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))
    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imshow("original: ", img)
    cv.imshow("undistorte ",dst)
    cv.waitKey(500)
    cv.imwrite(dst_dir+img_name.format(img_id), dst)
cv.destroyAllWindows()

In [39]:
cv.destroyAllWindows()